# Using fine-tuned function calling models in an e2e application

## Objective

Function calling only creates the call to an external API – it doesn’t execute it. To actually execute the request, you’ll need to extract the function name and arguments from the LLM response and proceed to call the function with those arguments. The function's output is in JSON format, which is then passed back to gpt-35-turbo to generate an appropriate result message for the user. 



## Time:

You should expect to sepnd 5-10 min running this sample.

### Before you begin
#### Installation
The following packages are required to execute this notebook.

In [ ]:
# Install the packages\
%pip install requests openai~=1.10 yfinance

Import the required packages.

In [ ]:
import json
import yfinance as yf
import pandas as pd
from openai import AzureOpenAI

### Parameters

In [ ]:
azure_endpoint = "https://<YOUR_RESOURCE_NAME>.openai.azure.com"
api_version = "2024-02-15-preview"
aoai_api_key = "<AOAI_RESOURCE_API_KEY>"

## Run the E2E example

Let's define the below two functions:

**get_current_stock_price(symbol):** This function takes a stock symbol as input, retrieves information about the current stock price using the yfinance library (yf), and returns a JSON-formatted string containing the symbol and current price.

**get_last_nday_stock_price(symbol, period):** This function takes a stock symbol and a period as inputs, retrieves historical stock price data for the specified period using the yfinance library (yf), converts the data to a Pandas DataFrame, and then converts the DataFrame to a JSON-formatted string before returning it.

In [ ]:
def get_current_stock_price(symbol: str) -> str:
    ticker = yf.Ticker(symbol).info
    info = {}
    if "currentPrice" in ticker:
        market_price = ticker["currentPrice"]
        info = {"symbol": symbol, "current_price": market_price}
    else:
        print(f"{symbol} is not valid.")
    return json.dumps(info)


def get_last_nday_stock_price(symbol: str, period: str) -> str:
    stock = yf.Ticker(symbol)
    data = stock.history(period=period)
    df = pd.DataFrame(data)
    return df.to_json(orient="records")

Define gpt_test function for inference and initialize an Azure OpenAI client for interaction with the Azure OpenAI service.

In [ ]:
def gpt_test() -> None:
    print("gpt_inference")


client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=aoai_api_key,
    api_version=api_version,
)

Lets declare function schema. This schema can contain multiple functions which can perform multiple intents. Our stock use case features two functions: the first one retrieves the current stock price, and the second one gets the stock price of last n days. 

For the hallucination use case, you should select the full function, whereas for the token reduction scenario, you should select the shortened function.

In [ ]:
# Full function for hallucination use case
functions = [
    {
        "name": "get_current_stock_price",
        "description": "Get the current stock price",
        "parameters": {
            "type": "object",
            "properties": {"symbol": {"type": "string", "description": "The stock symbol"}},
            "required": ["symbol"],
        },
    },
    {
        "name": "get_last_nday_stock_price",
        "description": "Get stock price last n days",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {"type": "string", "description": "The stock symbol"},
                "period": {"type": "string", "description": "Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max"},
            },
            "required": ["symbol", "period"],
        },
    },
]

In [ ]:
# short function for token reduction use case
functions = [
    {"name": "get_current_stock_price", "parameters": {"type": "object", "properties": {}, "required": ["symbol"]}},
    {
        "name": "get_last_nday_stock_price",
        "parameters": {"type": "object", "properties": {}, "required": ["symbol", "period"]},
    },
]

lets add the content of our test dataset into a list of strings.

In [ ]:
from pathlib import Path

# Replace '<provide path to test dataset>' with the actual path to your test dataset
file_path = Path("<provide path to test dataset>")

with file_path.open(errors="ignore") as json_file:
    json_list = list(json_file)

Now, we would like to use our fine-tuned function calling models in an e2e application. Here is the steps:
- Define the message with the question that you want to ask to GPT.
- Extract the system and user content from the JSON string.
- Create a list of messages to be sent to the Azure OpenAI model for completion.
- Send the messages for completion, setting the temperature as 0 to reduce randomness.
- Retrieve the function name and its arguments from the response_message object.
- Call the "get_current_stock_price" or "get_last_nday_stock_price" functions by passing those arguments.
- As the function’s output is still in JSON format, pass the JSON back to gpt-35-turbo model so that it can generate the appropriate result message for display to the user.

In [ ]:
mismatch_count = 0
for i, json_str in enumerate(json_list[:1]):
    print("starting on ", i)
    result = json.loads(json_str)
    if len(result["messages"]) > 2:
        system_content = result["messages"][0]["content"]
        user_content = result["messages"][1]["content"]
    else:
        user_content = result["messages"][0]["content"]

    messages = [
        {"role": "system", "content": system_content},
        # {"role": "user", "content": user_content},
        {"role": "user", "content": "what is the current price of Uber?"},  # token reduction
        # {"role": "user", "content": "What was the highest price that walmart's stock reached last quarter?"}, #token reduction
        # {"role": "user", "content": "What was the closing price of Titan Robotics' stock last Friday"}, #hallucination
    ]

    try:
        completion = client.chat.completions.create(
            model="<DEPLOYMENT_NAME>",
            messages=messages,
            temperature=0.0,  # to reduce randomness
            functions=functions,
            function_call="auto",
        )
        try:
            response_message = completion.choices[0].message
            print(completion.choices[0].message.model_dump_json(indent=2))

            # Check if the model wants to call a function
            function_calls = response_message.function_call
            if function_calls:
                function_name = response_message.function_call.name
                available_functions = {
                    "get_current_stock_price": get_current_stock_price,
                    "get_last_nday_stock_price": get_last_nday_stock_price,
                }
                function_to_call = available_functions[function_name]

                function_args = json.loads(response_message.function_call.arguments)
                function_response = function_to_call(**function_args)

                # Add the assistant response and function response to the messages
                messages.append(  # adding assistant response to messages
                    {
                        "role": response_message.role,
                        "function_call": {
                            "name": function_name,
                            "arguments": response_message.function_call.arguments,
                        },
                        "content": None,
                    }
                )
                messages.append(  # adding function response to messages
                    {
                        "role": "function",
                        "name": function_name,
                        "content": function_response,
                    }
                )

                # Call the API again to get the final response from the model
                second_response = client.chat.completions.create(
                    model="<DEPLOYMENT_NAME>",  # gpt-35-turbo-0613
                    messages=messages,
                )
                print(second_response.choices[0].message.content)

        except Exception as e:
            print("Error", i, completion)
            print(e)

    except Exception as e:
        print("Error", i)
        print(e)


if __name__ == "__main__":
    gpt_test()